# FYP: Swimming IMU Analysis Pipeline

**ML Analytics for Stroke Count, Technique and Fatigue Detection in Swimmers.**

We track **XYZ data** from accelerometer and gyroscope (IMU) on swimmers to get **good statistics** that help them:
- **Stroke count & rate** – pacing and consistency  
- **Stroke power** – effort per stroke  
- **Stroke reach** – efficiency (full reach = better)  
- **Hand entry angle** – technique (flat = good; inward = injury risk; outward = inefficient)  
- **Stroke Efficiency Index (SEI)** – reach vs power; drops when they fatigue  
- **Fatigue / form drop-off** – where performance declines during a swim  

All of this runs **without real data** using synthetic XYZ signals, so you can test the full pipeline in Colab before collecting Shimmer data.

---

**To run in Google Colab (no real data needed):**
1. **Upload the whole project folder** – all the `.py` files must be in the same place as this notebook (e.g. zip the project, upload to Colab, then unzip). Or clone from Git.
2. Run the cells in order. The first cells install packages and go to the project directory; then we generate **synthetic** accel/gyro data and run the full pipeline and show graphs.

## 1. Install dependencies

In [ ]:
# Run in Colab to install required packages
!pip install -q numpy pandas scipy scikit-learn matplotlib ruptures ahrs

**If you only have this notebook** (e.g. you opened it from GitHub and only the .ipynb is in Colab), run the cell below to clone the full project so all the `.py` files are available. Replace the URL with your repo URL.

In [ ]:
# If you opened only this notebook from GitHub, clone the full repo so the .py files are available.
# Set your repo URL and run this cell, then run the "Set project path" cell again.
import os
REPO_URL = "https://github.com/YourUsername/Final-Year-Project.git"  # <-- change to your repo
if not os.path.isfile("pipeline.py"):
    os.system(f"git clone {REPO_URL} /content/Final-Year-Project")
    os.chdir("/content/Final-Year-Project")
    print("Cloned project. Run the 'Set project path' cell above, then continue.")
else:
    print("Project files already present.")

## 2. Set project path (Colab)

In [ ]:
# Go to the project directory so imports (pipeline, synthetic_data, etc.) work.
# If you uploaded a ZIP, unzip first: !unzip -q -o /content/Final-Year-Project.zip
import os
if os.path.exists('/content/Final-Year-Project'):
    %cd /content/Final-Year-Project
elif os.path.exists('/content'):
    %cd /content
print("Working in:", os.getcwd())

In [ ]:
# Check that all project files are present (required for imports)
import os
required = ["pipeline.py", "synthetic_data.py", "config.py", "features.py", "signal_processing.py", "visualization.py"]
missing = [f for f in required if not os.path.isfile(f)]
if missing:
    print("Missing files:", missing)
    print("Upload the full project folder to Colab (all .py files + this notebook), then run this cell again.")
else:
    print("All project files found. You can run the pipeline below.")

## 3. Run pipeline on synthetic data (no real data yet)

In [ ]:
from pipeline import run_pipeline
from synthetic_data import generate_synthetic_imu
import config

# Generate 60 seconds of synthetic stroke-like data (50 SPM)
t, accel, gyro = generate_synthetic_imu(
    duration_sec=60.0,
    sample_rate_hz=config.DEFAULT_SAMPLE_RATE_HZ,
    stroke_rate_spm=50.0,
    seed=42,
)

result = run_pipeline(
    accel=accel,
    gyro=gyro,
    time_s=t,
    sample_rate_hz=config.DEFAULT_SAMPLE_RATE_HZ,
    mass_kg=70.0,
    use_fixed_reach=False,
)

print(f"Detected {len(result['df_features'])} strokes")
result['df_features'].head(10)

## 4. Show graphs

In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt

for fig in result['figures']:
    display(fig)

## 5. (Optional) Run on your own CSV

When you have Shimmer IMU data, upload the CSV and set the path below. If your column names differ from the default, use the `config` module or pass column names to `load_imu_csv`.

In [ ]:
# Example: load your CSV (after uploading to Colab)
# result = run_pipeline(filepath='/content/your_imu_export.csv', mass_kg=70, use_fixed_reach=True)
# result['df_features'].to_csv('extracted_features.csv', index=False)